origin：https://www.kaggle.com/code/eduardtrulls/imc-2023-submission-example?scriptVersionId=129165789

In [1]:
# General utilities
import os
from tqdm import tqdm
from time import time
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy

# CV/ML
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# 3D reconstruction
import pycolmap

os：Python的标准库，提供了与操作系统交互的功能，例如文件和目录操作。

tqdm：一个用于在循环中显示进度条的库。

time：Python的标准库，提供了与时间相关的功能，如计时和时间戳。

fastprogress：用于显示快速进度条的库。

gc：Python的标准库，提供了垃圾回收功能，用于自动管理内存。

numpy：一个用于科学计算的Python库，提供了高性能的多维数组对象和各种计算函数。

h5py：一个用于读写HDF5文件的库，HDF5是一种数据存储格式。

IPython.display：IPython的库，提供了在交互式环境中显示内容的功能。

collections.defaultdict：一个字典子类，提供了在访问不存在的键时返回默认值的功能。

copy.deepcopy：用于创建对象的深拷贝，包括嵌套对象的拷贝。

cv2：OpenCV库的Python接口，用于计算机视觉任务，如图像处理和计算机视觉算法。

torch：PyTorch深度学习框架的主要库，提供了张量操作和自动微分功能。

torch.nn.functional：PyTorch的子模块，提供了一些函数形式的神经网络操作，如激活函数和损失函数。

kornia：一个用于计算机视觉任务的PyTorch库，提供了各种几何变换和图像处理操作。

kornia.feature：kornia的子模块，提供了一些特征提取相关的函数和模块。

PIL.Image：Python Imaging Library（PIL）的模块，用于图像处理和操作。

timm：一个用于图像分类和计算机视觉任务的PyTorch库，提供了各种预训练模型和训练工具。

timm.data.resolve_data_config：timm库的函数，用于解析数据配置。

timm.data.transforms_factory.create_transform：timm库的函数，用于创建数据转换。

pycolmap：用于三维重建的Colmap库的Python接口，Colmap是一个开源的视觉SLAM系统。

In [2]:
print('Kornia version', K.__version__)
print('Pycolmap version', pycolmap.__version__)

LOCAL_FEATURE = 'KeyNetAffNetHardNet'
device=torch.device('cuda')
# Can be LoFTR, KeyNetAffNetHardNet, or DISK

#这两行代码分别给变量LOCAL_FEATURE和device赋值。
#LOCAL_FEATURE被赋值为字符串'KeyNetAffNetHardNet'，表示选择使用KeyNetAffNetHardNet作为本地特征算法。
#device被赋值为torch.device('cuda')，表示将使用CUDA加速来执行计算。

Kornia version 0.6.11
Pycolmap version 0.3.0


In [3]:
def arr_to_str(a):
    return ';'.join([str(x) for x in a.reshape(-1)])


def load_torch_image(fname, device=torch.device('cpu')):
    img = K.image_to_tensor(cv2.imread(fname), False).float() / 255.
    img = K.color.bgr_to_rgb(img.to(device))
    return img

函数首先使用OpenCV的cv2.imread函数加载图像文件，并使用K.image_to_tensor函数将图像转换为PyTorch张量。False参数表示图像不进行归一化。然后，将图像的像素值从BGR颜色空间转换为RGB颜色空间，使用K.color.bgr_to_rgb函数实现。最后，如果提供了device参数，将图像张量移动到指定的设备上，然后返回图像张量。

这两个函数分别用于将数组转换为字符串和加载图像并转换为PyTorch张量，为后续的数据处理和图像操作提供了方便的工具函数。

In [4]:
# We will use ViT global descriptor to get matching shortlists.
def get_global_desc(fnames, model,
                    device =  torch.device('cpu')):
    model = model.eval()
    model= model.to(device)
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    global_descs_convnext=[]
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        img = Image.open(img_fname_full).convert('RGB')
        timg = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            desc = model.forward_features(timg.to(device)).mean(dim=(-1,2))#
            #print (desc.shape)
            desc = desc.view(1, -1)
            desc_norm = F.normalize(desc, dim=1, p=2)
        #print (desc_norm)
        global_descs_convnext.append(desc_norm.detach().cpu())
    global_descs_all = torch.cat(global_descs_convnext, dim=0)
    return global_descs_all

#该函数用于提取图像的全局描述符。它接受一个图像文件名列表fnames，一个模型model，和一个可选的device参数，默认为torch.device('cpu')。
#首先，将模型设置为评估模式，并将其移动到指定的设备上。然后，使用resolve_data_config函数解析模型配置，
#并使用create_transform函数创建数据转换。然后，对于每个图像文件名，打开图像文件并将其转换为RGB格式，然后应用数据转换并将图像转换为张量。
#之后，使用模型提取特征，并计算特征的均值，得到全局描述符。最后，将所有图像的全局描述符拼接成一个张量并返回。

def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs
#该函数用于生成图像之间的完全匹配的索引对。它接受一个图像文件名列表img_fnames作为输入。
#函数通过两层循环遍历所有图像文件名的组合，生成索引对。对于索引i和j，其中i<j，会生成一个索引对(i, j)。最后，返回所有生成的索引对。

def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 20,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)

    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)

    model = timm.create_model('tf_efficientnet_b7',
                              checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b7/1/tf_efficientnet_b7_ra-6c08e654.pth')
    model.eval()
    descs = get_global_desc(fnames, model, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    # removing half
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

#该函数用于生成图像之间的匹配候选列表。它接受一个图像文件名列表fnames，以及一些可选的参数，如相似度阈值sim_th、
#最小匹配对数min_pairs、小于等于该值时采用完全匹配的图像数exhaustive_if_less和设备device，默认为torch.device('cpu')。
#首先，根据图像文件名列表的长度判断是否需要进行完全匹配。如果图像数小于等于exhaustive_if_less，
#则调用get_img_pairs_exhaustive函数生成完全匹配的索引对并返回。
#如果图像数大于exhaustive_if_less，则创建一个EfficientNet-B7模型，并将其设置为评估模式。
#然后，使用get_global_desc函数提取图像的全局描述符。使用欧氏距离计算描述符之间的距离矩阵，并将其转换为NumPy数组。
#接下来，根据相似度阈值将距离矩阵转换为二进制掩码。如果距离小于等于相似度阈值，则相应位置的掩码值为True，
#表示两个图像具有相似的描述符。然后，初始化一些变量，如计数器total、匹配列表matching_list和一个范围数组ar，用于生成索引。
#对于每个起始索引st_idx（从0到num_imgs-1），获取与该索引对应的掩码，得到相似的图像索引to_match。
#如果相似图像的数量少于最小匹配对数min_pairs，则选择距离最近的min_pairs个图像进行匹配。然后，遍历与起始索引匹配的图像索引，
#并判断它们之间的距离是否小于阈值1000。如果满足条件，则将索引对(st_idx, idx)添加到匹配列表中。最后，去重、排序匹配列表并返回。
#该函数用于生成匹配候选列表，根据全局描述符的相似度将图像进行匹配，返回一组匹配的索引对。

In [5]:
# Code to manipulate a colmap database.
# Forked from https://github.com/colmap/colmap/blob/dev/scripts/python/database.py

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

# This script is based on an original implementation by True Price.

import sys
import sqlite3
import numpy as np


IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

这段代码是一个用于操作COLMAP数据库的脚本，下面对其进行注释和解释：

COLMAPDatabase 类是一个继承自 sqlite3.Connection 的自定义类，用于操作 COLMAP 数据库。它包含了一系列方法来添加相机、图像、关键点、描述符、匹配等数据到数据库中。

IS_PYTHON3 变量用于检查是否运行在 Python 3 及以上的版本上。

MAX_IMAGE_ID 变量定义了图像ID的最大值，用于在生成匹配对ID时进行计算。

CREATE_CAMERAS_TABLE、CREATE_DESCRIPTORS_TABLE、CREATE_IMAGES_TABLE、CREATE_TWO_VIEW_GEOMETRIES_TABLE、CREATE_KEYPOINTS_TABLE、CREATE_MATCHES_TABLE 和 CREATE_NAME_INDEX 是用于创建数据库表的 SQL 语句。

image_ids_to_pair_id 函数用于将两个图像ID转换为匹配对ID。

pair_id_to_image_ids 函数用于将匹配对ID转换为两个图像ID。

array_to_blob 函数用于将数组转换为二进制数据。

blob_to_array 函数用于将二进制数据转换为数组。

add_camera 方法用于向数据库中添加相机信息。

add_image 方法用于向数据库中添加图像信息。

add_keypoints 方法用于向数据库中添加关键点信息。

add_descriptors 方法用于向数据库中添加描述符信息。

add_matches 方法用于向数据库中添加匹配信息。

add_two_view_geometry 方法用于向数据库中添加两视图几何信息。

该脚本提供了一组用于操作 COLMAP 数据库的方法，用于添加相机、图像、关键点、描述符、匹配等数据到数据库中，并提供了一些辅助函数用于数据转换和ID转换。

In [6]:
# Code to interface DISK with Colmap.
# Forked from https://github.com/cvlab-epfl/disk/blob/37f1f7e971cea3055bb5ccfc4cf28bfd643fa339/colmap/h5_to_db.py

#  Copyright [2020] [Michał Tyszkiewicz, Pascal Fua, Eduard Trulls]
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.

import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags


def get_focal(image_path, err_on_default=False):
    image         = Image.open(image_path)
    max_size      = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size
    
    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal
#get_focal 函数用于从图像文件中获取焦距信息。它接受图像文件路径 image_path 和一个可选的 err_on_default 参数，默认为 False。
#函数打开图像文件，并获取其尺寸信息。然后，尝试从图像的EXIF数据中获取焦距信息。如果成功获取到焦距，根据相对于35mm胶片的焦距进行换算，
#得到相对于图像尺寸的焦距。如果无法获取焦距信息，根据默认的先验值计算一个焦距。最后，返回计算得到的焦距。


def create_camera(db, image_path, camera_model):
    image         = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)
#create_camera 函数用于向数据库中添加相机信息。它接受一个数据库对象 db，图像文件路径 image_path 和相机模型 camera_model。
#函数打开图像文件，并获取图像的宽度和高度。然后，调用 get_focal 函数获取图像的焦距。根据相机模型类型，选择相应的相机模型ID和参数数组。
#最后，调用数据库的 add_camera 方法添加相机信息，并返回相机ID。

def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera = True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]

        fname_with_ext = filename# + img_ext
        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id
#add_keypoints 函数用于向数据库中添加关键点信息。它接受一个数据库对象 db，HDF5文件路径 h5_path，图像文件路径 image_path，
#图像文件扩展名 img_ext，相机模型 camera_model 和一个可选的 single_camera 参数，默认为 True。

#函数打开关键点的HDF5文件，并遍历文件中的每个关键点数据。对于每个关键点数据，构建关键点文件名（包括扩展名），
#并与图像文件路径进行拼接得到图像文件的完整路径。如果图像文件不存在，抛出异常。

#如果是第一次添加关键点数据或者不使用单一相机模型，调用 create_camera 函数创建相机，并返回相机ID。然后，
#调用数据库的 add_image 方法添加图像信息，并将图像文件名与图像ID进行映射保存。最后，调用数据库的 add_keypoints 方法添加关键点信息。

#返回关键点文件名到图像ID的映射字典 fname_to_id。

def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)
# 这段代码用于将DISK的结果与COLMAP进行接口对接，下面对其进行注释和解释：

def get_focal(image_path, err_on_default=False):
    image = Image.open(image_path)
    max_size = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size

    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal

#get_focal 函数用于从图像文件中获取焦距信息。它接受图像文件路径 image_path 和一个可选的 err_on_default 参数，默认为 False。

#函数打开图像文件，并获取其尺寸信息。然后，尝试从图像的EXIF数据中获取焦距信息。如果成功获取到焦距，根据相对于35mm胶片的焦距进行换算，得到相对于图像尺寸的焦距。如果无法获取焦距信息，根据默认的先验值计算一个焦距。最后，返回计算得到的焦距。

def create_camera(db, image_path, camera_model):
    image = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])

    return db.add_camera(model, width, height, param_arr)
#create_camera 函数用于向数据库中添加相机信息。它接受一个数据库对象 db，图像文件路径 image_path 和相机模型 camera_model。

#函数打开图像文件，并获取图像的宽度和高度。然后，调用 get_focal 函数获取图像的焦距。根据相机模型类型，选择相应的相机模型ID和参数数组。最后，调用数据库的 add_camera 方法添加相机信息，并返回相机ID。

def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera=True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]

        fname_with_ext = filename  # + img_ext
        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

#add_keypoints 函数用于向数据库中添加关键点信息。它接受一个数据库对象 db，HDF5文件路径 h5_path，图像文件路径 image_path，图像文件扩展名 img_ext，相机模型 camera_model 和一个可选的 single_camera 参数，默认为 True。

#函数打开关键点的HDF5文件，并遍历文件中的每个关键点数据。对于每个关键点数据，构建关键点文件名（包括扩展名），并与图像文件路径进行拼接得到图像文件的完整路径。如果图像文件不存在，抛出异常。

#如果是第一次添加关键点数据或者不使用单一相机模型，调用 create_camera 函数创建相机，并返回相机ID。然后，调用数据库的 add_image 方法添加图像信息，并将图像文件名与图像ID进行映射保存。最后，调用数据库的 add_keypoints 方法添加关键点信息。

#返回关键点文件名到图像ID的映射字典 fname_to_id。


def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')

    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue

                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)
#add_matches 函数用于向数据库中添加匹配信息。它接受一个数据库对象 db，HDF5文件路径 h5_path 和图像文件名到图像ID的映射字典 fname_to_id。

#函数打开匹配的HDF5文件，并遍历文件中的每对匹配数据。对于每对匹配数据，获取对应的图像ID。使用图像ID计算匹配对ID，
#并检查是否已经添加过该匹配对。如果已经添加过，则发出警告并跳过。

#如果匹配对没有被添加过，则获取匹配数据，并调用数据库的 add_matches 方法添加匹配信息。

#最后，更新进度条并完成匹配添加。

#该代码用于将DISK的关键点和匹配结果添加到COLMAP的数据库中，以便进行后续的三维重建。



In [7]:
# Making kornia local features loading w/o internet
class KeyNetAffNetHardNet(KF.LocalFeature):
    """Convenience module, which implements KeyNet detector + AffNet + HardNet descriptor.

    .. image:: _static/img/keynet_affnet.jpg
    """

    def __init__(
        self,
        num_features: int = 5000,
        upright: bool = False,
        device = torch.device('cpu'),
        scale_laf: float = 1.0,
    ):
        ori_module = KF.PassLAF() if upright else KF.LAFOrienter(angle_detector=KF.OriNet(False)).eval()
        if not upright:
            weights = torch.load('/kaggle/input/kornia-local-feature-weights/OriNet.pth')['state_dict']
            ori_module.angle_detector.load_state_dict(weights)
        detector = KF.KeyNetDetector(
            False, num_features=num_features, ori_module=ori_module, aff_module=KF.LAFAffNetShapeEstimator(False).eval()
        ).to(device)
        kn_weights = torch.load('/kaggle/input/kornia-local-feature-weights/keynet_pytorch.pth')['state_dict']
        detector.model.load_state_dict(kn_weights)
        affnet_weights = torch.load('/kaggle/input/kornia-local-feature-weights/AffNet.pth')['state_dict']
        detector.aff.load_state_dict(affnet_weights)
        
        hardnet = KF.HardNet(False).eval()
        hn_weights = torch.load('/kaggle/input/kornia-local-feature-weights/HardNetLib.pth')['state_dict']
        hardnet.load_state_dict(hn_weights)
        descriptor = KF.LAFDescriptor(hardnet, patch_size=32, grayscale_descriptor=True).to(device)
        super().__init__(detector, descriptor, scale_laf)

这段代码定义了一个名为KeyNetAffNetHardNet的类，继承自KF.LocalFeature类。它实现了KeyNet检测器 + AffNet + HardNet描述符的功能。

构造函数__init__接受一些参数，包括num_features（特征数量）、upright（是否使用直立（upright）的检测器，默认为False）、device（设备，默认为torch.device('cpu')）和scale_laf（LAF（局部仿射帧）的尺度，默认为1.0）。

在构造函数中，首先根据upright参数选择使用KF.PassLAF()或KF.LAFOrienter(angle_detector=KF.OriNet(False)).eval()作为角度检测器。如果不是直立的检测器，加载预训练的角度检测器模型的权重。

然后，创建KF.KeyNetDetector对象，设置其参数，并加载预训练的KeyNet模型和AffNet模型的权重。

接下来，创建KF.HardNet对象，并加载预训练的HardNet模型的权重。

最后，创建KF.LAFDescriptor对象，设置其参数，并将HardNet模型作为描述符。

该类封装了KeyNet检测器、AffNet形状估计器和HardNet描述符，方便使用并提供局部特征提取功能。

In [8]:
def detect_features(img_fnames,
                    num_feats = 4096,
                    upright = False,
                    device=torch.device('cpu'),
                    feature_dir = '.featureout',
                    resize_small_edge_to = 600):
    if LOCAL_FEATURE == 'DISK':
        # Load DISK from Kaggle models so it can run when the notebook is offline.
        disk = KF.DISK().to(device)
        pretrained_dict = torch.load('/kaggle/input/disk/pytorch/depth-supervision/1/loftr_outdoor.ckpt', map_location=device)
        disk.load_state_dict(pretrained_dict['extractor'])
        disk.eval()
    if LOCAL_FEATURE == 'KeyNetAffNetHardNet':
        feature = KeyNetAffNetHardNet(num_feats, upright, device).to(device).eval()
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/lafs.h5', mode='w') as f_laf, \
         h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in progress_bar(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                timg = load_torch_image(img_path, device=device)
                H, W = timg.shape[2:]
                if resize_small_edge_to is None:
                    timg_resized = timg
                else:
                    timg_resized = K.geometry.resize(timg, resize_small_edge_to, antialias=True)
                    print(f'Resized {timg.shape} to {timg_resized.shape} (resize_small_edge_to={resize_small_edge_to})')
                h, w = timg_resized.shape[2:]
                if LOCAL_FEATURE == 'DISK':
                    features = disk(timg_resized, num_feats, pad_if_not_divisible=True)[0]
                    kps1, descs = features.keypoints, features.descriptors
                    
                    lafs = KF.laf_from_center_scale_ori(kps1[None], torch.ones(1, len(kps1), 1, 1, device=device))
                if LOCAL_FEATURE == 'KeyNetAffNetHardNet':
                    lafs, resps, descs = feature(K.color.rgb_to_grayscale(timg_resized))
                lafs[:,:,0,:] *= float(W) / float(w)
                lafs[:,:,1,:] *= float(H) / float(h)
                desc_dim = descs.shape[-1]
                kpts = KF.get_laf_center(lafs).reshape(-1, 2).detach().cpu().numpy()
                descs = descs.reshape(-1, desc_dim).detach().cpu().numpy()
                f_laf[key] = lafs.detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

def match_features(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=15, 
                   force_mutual = True,
                   matching_alg='smnn'
                  ):
    assert matching_alg in ['smnn', 'adalam']
    with h5py.File(f'{feature_dir}/lafs.h5', mode='r') as f_laf, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:

        for pair_idx in progress_bar(index_pairs):
                    idx1, idx2 = pair_idx
                    fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
                    key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
                    lafs1 = torch.from_numpy(f_laf[key1][...]).to(device)
                    lafs2 = torch.from_numpy(f_laf[key2][...]).to(device)
                    desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
                    desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
                    if matching_alg == 'adalam':
                        img1, img2 = cv2.imread(fname1), cv2.imread(fname2)
                        hw1, hw2 = img1.shape[:2], img2.shape[:2]
                        adalam_config = KF.adalam.get_adalam_default_config()
                        #adalam_config['orientation_difference_threshold'] = None
                        #adalam_config['scale_rate_threshold'] = None
                        adalam_config['force_seed_mnn']= False
                        adalam_config['search_expansion'] = 16
                        adalam_config['ransac_iters'] = 128
                        adalam_config['device'] = device
                        dists, idxs = KF.match_adalam(desc1, desc2,
                                                      lafs1, lafs2, # Adalam takes into account also geometric information
                                                      hw1=hw1, hw2=hw2,
                                                      config=adalam_config) # Adalam also benefits from knowing image size
                    else:
                        dists, idxs = KF.match_smnn(desc1, desc2, 0.98)
                    if len(idxs)  == 0:
                        continue
                    # Force mutual nearest neighbors
                    if force_mutual:
                        first_indices = get_unique_idxs(idxs[:,1])
                        idxs = idxs[first_indices]
                        dists = dists[first_indices]
                    n_matches = len(idxs)
                    if False:
                        print (f'{key1}-{key2}: {n_matches} matches')
                    group  = f_match.require_group(key1)
                    if n_matches >= min_matches:
                         group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return

def match_loftr(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout_loftr',
                   device=torch.device('cpu'),
                   min_matches=15, resize_to_ = (640, 480)):
    matcher = KF.LoFTR(pretrained=None)
    matcher.load_state_dict(torch.load('/kaggle/input/loftr/pytorch/outdoor/1/loftr_outdoor.ckpt')['state_dict'])
    matcher = matcher.to(device).eval()

    # First we do pairwise matching, and then extract "keypoints" from loftr matches.
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='w') as f_match:
        for pair_idx in progress_bar(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            # Load img1
            timg1 = K.color.rgb_to_grayscale(load_torch_image(fname1, device=device))
            H1, W1 = timg1.shape[2:]
            if H1 < W1:
                resize_to = resize_to_[1], resize_to_[0]
            else:
                resize_to = resize_to_
            timg_resized1 = K.geometry.resize(timg1, resize_to, antialias=True)
            h1, w1 = timg_resized1.shape[2:]

            # Load img2
            timg2 = K.color.rgb_to_grayscale(load_torch_image(fname2, device=device))
            H2, W2 = timg2.shape[2:]
            if H2 < W2:
                resize_to2 = resize_to[1], resize_to[0]
            else:
                resize_to2 = resize_to_
            timg_resized2 = K.geometry.resize(timg2, resize_to2, antialias=True)
            h2, w2 = timg_resized2.shape[2:]
            with torch.inference_mode():
                input_dict = {"image0": timg_resized1,"image1": timg_resized2}
                correspondences = matcher(input_dict)
            mkpts0 = correspondences['keypoints0'].cpu().numpy()
            mkpts1 = correspondences['keypoints1'].cpu().numpy()

            mkpts0[:,0] *= float(W1) / float(w1)
            mkpts0[:,1] *= float(H1) / float(h1)

            mkpts1[:,0] *= float(W2) / float(w2)
            mkpts1[:,1] *= float(H2) / float(h2)

            n_matches = len(mkpts1)
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=np.concatenate([mkpts0, mkpts1], axis=1))

    # Let's find unique loftr pixels and group them together.
    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1
    
    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
    return

def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

这段代码实现了一系列函数用于进行特征检测、特征匹配和将特征导入到COLMAP数据库中。

detect_features函数用于检测图像中的特征。它接受一系列图像文件名作为输入，然后使用指定的局部特征提取算法（如DISK或KeyNetAffNetHardNet）提取每个图像的局部特征。提取的特征包括局部仿射帧（LAF）、关键点坐标和描述符。提取的特征将保存在指定的特征目录中的HDF5文件中。

get_unique_idxs函数用于获取张量中唯一元素的第一次出现的索引。

match_features函数用于特征匹配。它接受图像文件名列表和图像对的索引对作为输入。对于每对图像，它从特征目录中加载对应的局部特征，并使用指定的匹配算法（如SMNN或Adalam）进行特征匹配。匹配的结果将保存在特征目录中的HDF5文件中。

match_loftr函数是特定于LoFTR模型的特征匹配函数。它使用LoFTR模型进行特征匹配，并将匹配结果保存在特征目录中的HDF5文件中。

import_into_colmap函数用于将特征导入到COLMAP数据库中。它连接到数据库，创建必要的表格，并将特征从特征目录中的HDF5文件导入到数据库中。

这些函数可以按照特定的顺序调用，以便进行特征检测、特征匹配和导入到COLMAP数据库的完整流程

In [9]:
src = '/kaggle/input/image-matching-challenge-2023'

In [10]:
# Get data from csv.

data_dict = {}
with open(f'{src}/sample_submission.csv', 'r') as f:
    for i, l in enumerate(f):
        # Skip header.
        if l and i > 0:
            image, dataset, scene, _, _ = l.strip().split(',')
            if dataset not in data_dict:
                data_dict[dataset] = {}
            if scene not in data_dict[dataset]:
                data_dict[dataset][scene] = []
            data_dict[dataset][scene].append(image)

这段代码从CSV文件中获取数据，并将其存储在data_dict字典中。

In [11]:
for dataset in data_dict:
    for scene in data_dict[dataset]:
        print(f'{dataset} / {scene} -> {len(data_dict[dataset][scene])} images')

2cfa01ab573141e4 / 2fa124afd1f74f38 -> 3 images


该代码段用于遍历data_dict字典，打印每个数据集和场景中图像的数量。它会输出每个数据集和场景的名称以及对应的图像数量。这样可以对数据集和场景中的图像数量进行可视化和统计。

In [12]:
out_results = {}
timings = {"shortlisting":[],
           "feature_detection": [],
           "feature_matching":[],
           "RANSAC": [],
           "Reconstruction": []}

out_results是一个空的字典，用于存储输出结果。

timings是一个包含不同阶段时间统计的字典。它包含以下键：

"shortlisting"：用于存储图像筛选阶段的时间。
"feature_detection"：用于存储特征检测阶段的时间。
"feature_matching"：用于存储特征匹配阶段的时间。
"RANSAC"：用于存储RANSAC阶段的时间。
"Reconstruction"：用于存储重建阶段的时间。
这些键对应的值是空列表，用于存储每个阶段的时间统计。

In [13]:
# Function to create a submission file.
def create_submission(out_results, data_dict):
    with open(f'submission.csv', 'w') as f:
        f.write('image_path,dataset,scene,rotation_matrix,translation_vector\n')
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print (image)
                        R = scene_res[image]['R'].reshape(-1)
                        T = scene_res[image]['t'].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    f.write(f'{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n')

create_submission是一个用于创建提交文件的函数。

它接受两个参数：

out_results：存储了匹配和重建结果的字典。
data_dict：存储了待处理图像的数据字典。
函数会打开一个名为submission.csv的文件，并写入CSV文件的头部信息。然后，它会遍历data_dict中的每个数据集和场景，并检查out_results中是否有对应的结果。如果有结果，则从结果中提取旋转矩阵和平移向量。如果没有结果，则使用单位矩阵和零向量作为默认值。最后，将图像路径、数据集、场景、旋转矩阵和平移向量写入CSV文件中。

请注意，arr_to_str是一个将NumPy数组转换为字符串的辅助函数。

In [14]:
gc.collect()
datasets = []
for dataset in data_dict:
    datasets.append(dataset)

for dataset in datasets:
    print(dataset)
    if dataset not in out_results:
        out_results[dataset] = {}
    for scene in data_dict[dataset]:
        print(scene)
        # Fail gently if the notebook has not been submitted and the test data is not populated.
        # You may want to run this on the training data in that case?
        img_dir = f'{src}/test/{dataset}/{scene}/images'
        if not os.path.exists(img_dir):
            continue
        # Wrap the meaty part in a try-except block.
        try:
            out_results[dataset][scene] = {}
            img_fnames = [f'{src}/test/{x}' for x in data_dict[dataset][scene]]
            print (f"Got {len(img_fnames)} images")
            feature_dir = f'featureout/{dataset}_{scene}'
            if not os.path.isdir(feature_dir):
                os.makedirs(feature_dir, exist_ok=True)
            t=time()
            index_pairs = get_image_pairs_shortlist(img_fnames,
                                  sim_th = 0.7321971, # should be strict
                                  min_pairs =37, # we select at least min_pairs PER IMAGE with biggest similarity
                                  exhaustive_if_less = 20,
                                  device=device)
            t=time() -t 
            timings['shortlisting'].append(t)
            print (f'{len(index_pairs)}, pairs to match, {t:.4f} sec')
            gc.collect()
            t=time()
            if LOCAL_FEATURE != 'LoFTR':
                detect_features(img_fnames, 
                                8192,
                                feature_dir=feature_dir,
                                upright=True,
                                device=device,
                                resize_small_edge_to=800
                               )
                gc.collect()
                t=time() -t 
                timings['feature_detection'].append(t)
                print(f'Features detected in  {t:.4f} sec')
                t=time()
                match_features(img_fnames, index_pairs, feature_dir=feature_dir,device=device)
            else:
                match_loftr(img_fnames, index_pairs, feature_dir=feature_dir, device=device, resize_to_=(600, 800))
            t=time() -t 
            timings['feature_matching'].append(t)
            print(f'Features matched in  {t:.4f} sec')
            database_path = f'{feature_dir}/colmap.db'
            if os.path.isfile(database_path):
                os.remove(database_path)
            gc.collect()
            import_into_colmap(img_dir, feature_dir=feature_dir,database_path=database_path)
            output_path = f'{feature_dir}/colmap_rec_{LOCAL_FEATURE}'

            t=time()
            pycolmap.match_exhaustive(database_path)
            t=time() - t 
            timings['RANSAC'].append(t)
            print(f'RANSAC in  {t:.4f} sec')

            t=time()
            # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
            mapper_options = pycolmap.IncrementalMapperOptions()
            mapper_options.min_model_size = 3
            os.makedirs(output_path, exist_ok=True)
            maps = pycolmap.incremental_mapping(database_path=database_path, image_path=img_dir, output_path=output_path, options=mapper_options)
            print(maps)
            #clear_output(wait=False)
            t=time() - t
            timings['Reconstruction'].append(t)
            print(f'Reconstruction done in  {t:.4f} sec')
            imgs_registered  = 0
            best_idx = None
            print ("Looking for the best reconstruction")
            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print (idx1, rec.summary())
                    if len(rec.images) > imgs_registered:
                        imgs_registered = len(rec.images)
                        best_idx = idx1
            if best_idx is not None:
                print (maps[best_idx].summary())
                for k, im in maps[best_idx].images.items():
                    key1 = f'{dataset}/{scene}/images/{im.name}'
                    out_results[dataset][scene][key1] = {}
                    out_results[dataset][scene][key1]["R"] = deepcopy(im.rotmat())
                    out_results[dataset][scene][key1]["t"] = deepcopy(np.array(im.tvec))
            print(f'Registered: {dataset} / {scene} -> {len(out_results[dataset][scene])} images')
            print(f'Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images')
            create_submission(out_results, data_dict)
            gc.collect()
        except:
            pass

2cfa01ab573141e4
2fa124afd1f74f38


该代码段中的循环遍历数据集和场景，并在每个场景中执行以下操作：

检查输出结果字典out_results中是否存在当前数据集和场景的条目，如果不存在，则创建空字典作为条目。
获取图像文件路径列表img_fnames。
创建特征存储目录feature_dir。
进行图像对的筛选，得到用于匹配的图像对索引index_pairs。
检测特征并进行匹配。
如果使用的是非LoFTR本地特征，则先进行特征检测，然后进行特征匹配。
如果使用的是LoFTR本地特征，则直接进行特征匹配。
创建Colmap数据库并将特征导入。
执行RANSAC算法进行几何一致性估计。
执行重建过程，生成稀疏点云。
查找生成的重建中包含最多图像的模型，并提取该模型中每个图像的旋转矩阵和平移向量。
将图像的旋转矩阵和平移向量存储到out_results字典中。
创建提交文件。
请注意，代码中使用了try-except块来捕获任何可能的错误，并在出现错误时继续执行下一个场景的处理。这样可以确保即使在处理某些场景时出现问题，代码也能继续运行并处理其他场景。

另外，gc.collect()语句用于手动触发垃圾回收，以释放内存。timings字典用于记录每个步骤的运行时间。

In [15]:
create_submission(out_results, data_dict)

create_submission函数用于创建提交文件，将out_results中的结果以CSV格式写入到submission.csv文件中。

函数的输入参数包括out_results和data_dict。

out_results是一个字典，包含了每个数据集和场景中的图像的旋转矩阵和平移向量信息。

data_dict是一个字典，包含了每个数据集和场景中的图像文件路径信息。

函数会遍历data_dict中的每个数据集和场景，并在submission.csv文件中写入每个图像的信息，包括图像路径、数据集、场景、旋转矩阵和平移向量。

函数的输出是一个submission.csv文件，其中包含了所有图像的信息，可以用于提交比赛结果。